In [2]:
from datetime import date, timedelta
from dateutil.rrule import rrule, MONTHLY
import ipywidgets as ipw

In [3]:
##### THE FOLLOWING SECTION SHOULD BE TOUCHED ONLY TO UPDATE IF E.G. THE LAW CHANGES

socialCostsPerc      = 32.1     ### percentage for social costs
daysPerYearSeverance = 20       ### # of days per year for severance after firing

minimumSalary = {
        'Predoctoral' : 1929.00,
        'Postdoctoral': 2302.40,
}

In [4]:
@ipw.interact(
    startdate = ipw.DatePicker(value=date.today(), description="Starting date"),
    enddate   = ipw.DatePicker(value=date.today(), description="End date"),
    hiringProfile = ipw.Dropdown(options=["Predoctoral", "Postdoctoral"], description="hiring profile"),
    grossSalaryMonth = ipw.FloatText(value=minimumSalary["Predoctoral"], description="Gross salary")
)
def print_costs(startdate,enddate,hiringProfile,grossSalaryMonth):
    if grossSalaryMonth < minimumSalary[hiringProfile]:
        print(f"Error! Minimum salary for hiring profile {hiringProfile} is {minimumSalary[hiringProfile]:.2f}€/month")
        return

    total = 0
    for year in range(startdate.year, enddate.year+1):
        start_in_year = max(startdate, date(year, 1, 1))
        end_in_year = min(enddate, date(year, 12, 31))
        end_p1 = end_in_year + timedelta(days=1)

        month_limits = [d.date() for d in rrule(MONTHLY, dtstart=start_in_year, until=end_p1)]
        full_months = len(month_limits) - 1

        extra_days = (end_in_year + timedelta(days=1) - month_limits[-1]).days
        grossSalary = grossSalaryMonth * (full_months + extra_days * 12 / 365)
        hiringCosts = grossSalary * (1 + 0.01 * socialCostsPerc)
        total       += hiringCosts

        print(f"{year:9d}: {hiringCosts:8.2f}")

    ##### compute severance and total
    nDays               = (enddate - startdate).days
    nDaysSeverance      = nDays / 365 * daysPerYearSeverance
    equivalentSalaryDay = grossSalaryMonth * 12 / 365
    severance           = nDaysSeverance * equivalentSalaryDay
    total              += severance
    print(f"Severance: {severance:8.2f}")
    print(f"    Total: {total:8.2f}")

interactive(children=(DatePicker(value=datetime.date(2025, 6, 19), description='Starting date', step=1), DateP…